In [1]:
import pandas as pd;
import numpy as np;
import random;
from faker import Faker
import hashlib
import os
import pathlib
script_dir = pathlib.Path().resolve()
os.chdir(script_dir)

In [2]:
# Function for converting counts to proportions
def props(data,col):
    data['props']=data.groupby(col)['COUNT'].transform(lambda x: x/sum(x))
    return data

In [3]:
# Function for selecting a language based off of a country
def selectingLanguage(country):
        selectedCountry=country_language_prop_dict.get(country,"")
        if selectedCountry=="":
            return 'en'
        else:
            return np.random.choice(selectedCountry['languages'],p=selectedCountry['props'])

In [ ]:
#Joining through sampling other dataframe on name
def joining(data, otherData):
    sampled_rows = data.groupby('name', group_keys=False).apply(
        lambda group: otherData[otherData['name'] == group['name'].iloc[0]]
        .sample(n=len(group), replace=True, weights='props')
        .reset_index(drop=True)
    ).reset_index(drop=True)

    sampled_rows = sampled_rows.drop(columns=['name'], errors='ignore')
    data = data.reset_index(drop=True)
    data = pd.concat([data, sampled_rows], axis=1)
    return data

In [ ]:
#Joining through sampling on states
def joining_states(data,otherData,col):
    data[col]=data.groupby('name', group_keys=False).apply(
        lambda group: np.where(
            group['countryCode'] == 'US',
            np.random.choice(
                otherData[otherData['name'] == group['name'].iloc[0]][col],
                size=len(group),
                replace=True,
                p=otherData[otherData['name'] == group['name'].iloc[0]]['props']
            ),np.nan)
        ).explode().reset_index(drop=True)
    return data

In [ ]:
# Creating hashed_ids
def hash_id(value): 
    return hashlib.sha256(str(value).encode()).hexdigest()[:10]

In [ ]:
# Ratings creation based off generation
def ratings_generators(row):
    mean, std = rating.get(row['generation'], rating['None']).get(row['gender'], (60, 15))
    return np.clip(np.random.normal(loc=mean, scale=std), 0, 100)

In [ ]:
# Simulating birth dates
def generate_birth_date(generation):
    if generation in generations_to_years and generations_to_years[generation]:
        start_year, end_year = generations_to_years[generation]
        year = random.randint(start_year, end_year)
        month = random.randint(1, 12)
        if month == 2:
            if year%4==0:
                day=random.randint(1, 29)
            else:
                day = random.randint(1, 28)
        elif month in [1,3,5,7,8,10,12]:
            day = random.randint(1, 31)
        else:
            day=random.randint(1,30)
        return f"{year}-{month:02d}-{day:02d}"

In [ ]:
fake=Faker()

In [ ]:
# Reading in counts
country_data=pd.read_csv("Count_of_Game_Name_by_Country_Code.csv")
gender_data=pd.read_csv("Count_of_Game_Name_by_Gender.csv")
language_data=pd.read_csv("Count_of_Game_Name_by_Language.csv")
platform_data=pd.read_csv("Count_of_Game_Name_by_Platform_and_Player_Generation.csv")
state_data=pd.read_csv("Count_of_Game_Name_by_US_State.csv")

In [ ]:
# renaming column names of platform data
platform_data.columns=['name','platform','generation','COUNT']

In [ ]:
# Converting counts to proportions for each read in dataset

gender_data=props(gender_data,'name')
language_data=props(language_data,'name')
country_data=props(country_data,'name')
state_data=props(state_data,'name')
platform_data=props(platform_data,'name')

In [ ]:
# Expanding count of rows into observations
data=platform_data.loc[platform_data.index.repeat(platform_data['COUNT'])].reset_index(drop=True)
country_unaggregated=country_data.loc[country_data.index.repeat(country_data['COUNT'])].reset_index(drop=True)
state_unaggregated=state_data.loc[state_data.index.repeat(state_data['COUNT'])].reset_index(drop=True)
language_unaggregated=language_data.loc[language_data.index.repeat(language_data['COUNT'])].reset_index(drop=True)
platform_unaggregated=platform_data.loc[platform_data.index.repeat(platform_data['COUNT'])].reset_index(drop=True)

In [ ]:
# Dropping unnecessary columns
data.drop('COUNT',axis=1,inplace=True)
country_unaggregated.drop('COUNT',axis=1,inplace=True)
state_unaggregated.drop('COUNT',axis=1,inplace=True)
language_unaggregated.drop('COUNT',axis=1,inplace=True)

In [ ]:
#Mapping of Country to languages. Based off common languages spoken in country.
country_to_languages = {
    'AE': ['ar'],
    'AF': ['fa', 'ps'],
    'AL': ['sq'],
    'AM': ['hy'],
    'AN': ['nl'],
    'AQ': ['en'],
    'AR': ['es'],
    'AS': ['sm'],
    'AT': ['de'],
    'AU': ['en'],
    'AW': ['nl'],
    'AZ': ['az'],
    'BA': ['bs', 'sr'],
    'BB': ['en'],
    'BD': ['bn'],
    'BE': ['nl', 'fr', 'de'],
    'BG': ['bg'],
    'BH': ['ar'],
    'BJ': ['fr'],
    'BM': ['en'],
    'BO': ['es'],
    'BR': ['pt'],
    'BT': ['dz'],
    'BY': ['be', 'ru'],
    'BZ': ['en'],
    'CA': ['en', 'fr'],
    'CH': ['de', 'fr', 'rm'],
    'CL': ['es'],
    'CN': ['zh'],
    'CO': ['es'],
    'CR': ['es'],
    'CS': ['sr'],
    'CX': ['en'],
    'CY': ['el'],
    'CZ': ['cs'],
    'DE': ['de'],
    'DK': ['da'],
    'DO': ['es'],
    'DZ': ['ar'],
    'EC': ['es'],
    'EE': ['et'],
    'EG': ['ar'],
    'ES': ['es', 'ca'],
    'FI': ['fi', 'sv'],
    'FM': ['en'],
    'FR': ['fr'],
    'GA': ['fr'],
    'GB': ['en'],
    'GD': ['en'],
    'GE': ['ka'],
    'GG': ['en'],
    'GH': ['en'],
    'GP': ['fr'],
    'GR': ['el'],
    'GS': ['en'],
    'GT': ['es'],
    'GU': ['en'],
    'HK': ['zh', 'en'],
    'HM': ['en'],
    'HN': ['es'],
    'HR': ['hr'],
    'HU': ['hu'],
    'ID': ['id'],
    'IE': ['en'],
    'IL': ['he'],
    'IN': ['hi', 'en', 'bn', 'te', 'ta', 'ml', 'pa'],
    'IQ': ['ar', 'ku'],
    'IS': ['is'],
    'IT': ['it'],
    'JE': ['en'],
    'JM': ['en'],
    'JO': ['ar'],
    'JP': ['ja'],
    'KE': ['sw', 'en'],
    'KR': ['ko'],
    'KW': ['ar'],
    'KZ': ['ru'],
    'LB': ['ar'],
    'LK': ['si', 'ta'],
    'LT': ['lt'],
    'LU': ['lb', 'fr', 'de'],
    'LV': ['lv'],
    'LY': ['ar'],
    'MA': ['ar', 'fr'],
    'MC': ['fr'],
    'MD': ['ro'],
    'MK': ['mk'],
    'MP': ['en'],
    'MQ': ['fr'],
    'MT': ['mt'],
    'MU': ['fr'],
    'MV': ['dv'],
    'MX': ['es'],
    'MY': ['ms'],
    'MZ': ['pt'],
    'NG': ['en'],
    'NI': ['es'],
    'NL': ['nl'],
    'NO': ['no', 'nb', 'nn'],
    'NR': ['en'],
    'NZ': ['en', 'mi'],
    'OM': ['ar'],
    'PA': ['es'],
    'PE': ['es'],
    'PH': ['tl', 'en'],
    'PK': ['ur'],
    'PL': ['pl'],
    'PM': ['fr'],
    'PN': ['en'],
    'PR': ['es'],
    'PT': ['pt'],
    'PY': ['es'],
    'QA': ['ar'],
    'RE': ['fr'],
    'RO': ['ro'],
    'RS': ['sr'],
    'RU': ['ru'],
    'SA': ['ar'],
    'SE': ['sv'],
    'SG': ['en', 'ms', 'zh', 'ta'],
    'SI': ['sl'],
    'SK': ['sk'],
    'SV': ['es'],
    'TG': ['fr'],
    'TH': ['th'],
    'TM': ['tk'],
    'TN': ['ar'],
    'TR': ['tr'],
    'TT': ['en'],
    'TW': ['zh'],
    'UA': ['uk', 'ru'],
    'UG': ['en', 'sw'],
    'UM': ['en'],
    'US': ['en'],
    'UY': ['es'],
    'VC': ['en'],
    'VE': ['es'],
    'VI': ['en'],
    'VN': ['vi'],
    'ZA': ['af', 'en', 'zu'],
    'ZM': ['en'],
    'ZW': ['en']

}


In [ ]:
#Creating dataframe of country_to_languages dictonary
country_to_languages=pd.DataFrame.from_dict(country_to_languages, orient='index')
country_to_languages.columns = [f'Language_{i+1}' for i in range(country_to_languages.shape[1])]
country_to_languages.insert(0,'Country',country_to_languages.index)
country_to_languages.reset_index(drop=True, inplace=True)

In [ ]:
#Prep for joining
country_unaggregated.columns=['countryCode','name','country_props']
country_language=country_unaggregated.merge(country_to_languages,how='left',left_on='countryCode',right_on='Country')
language_unaggregated=language_unaggregated.groupby(['name','language'])['props'].first().reset_index()

In [ ]:
#Actual joining of countries to languages based off the dictionary
language_cols = ["Language_1", "Language_2", "Language_3", "Language_4", "Language_5", "Language_6", "Language_7"]

for lang in language_cols:
    country_language = country_language.merge(
        language_unaggregated,
        how='left',
        left_on=['name', lang],
        right_on=['name', 'language'],
        suffixes=("", f"_{lang}")
    ).drop(columns=["language"])


In [ ]:
# Filling all missing props to 0 for countries that have less than 7 official languages
columns_props = ['props', 'props_Language_2', 'props_Language_3', 'props_Language_4', 'props_Language_5', 'props_Language_6', 'props_Language_7']
country_language[columns_props] = country_language[columns_props].fillna(0)


In [ ]:
#Normalizing props of languages to sum up to one
prop_cols=['props','props_Language_2','props_Language_3','props_Language_4','props_Language_5','props_Language_6','props_Language_7']
country_language[prop_cols]=country_language[prop_cols].div(country_language[prop_cols].sum(axis=1), axis=0)

In [ ]:
#Replacing missing languages with mode language of English
country_language['Language_1']=country_language['Language_1'].fillna('en')
country_language['props']=country_language['props'].fillna(1.0)
country_language=country_language.rename(columns={'props':'props_Language_1'})

In [ ]:
#Creating a dictionary for country to languages with proportions 
country_language_dict=country_language.drop(['countryCode','name','country_props'],axis=1)
country_language_dict=country_language_dict.drop_duplicates().reset_index(drop=True)
country_language_prop_dict = {}
for _, row in country_language_dict.iterrows():
    country = row['Country']
    languages = [row[f'Language_{i}'] for i in range(1, 8) if pd.notna(row[f'Language_{i}']) and row[f'Language_{i}'] is not None]
    prop = [row[f'props_Language_{i}'] for i in range(1, 8) if pd.notna(row[f'Language_{i}']) and row[f'Language_{i}'] is not None]

    country_language_prop_dict[country] = {
        'languages': languages,
        'props': prop
    }

In [ ]:
#Creating new dataframe with only necessary features
country_language=country_language[['countryCode','name','country_props']]


In [ ]:
# Selecting language based off country code
country_language['Language']=country_language['countryCode'].apply(selectingLanguage)

In [ ]:
#Preping country_language data to be joined with other data
country_language.columns=['countryCode','name','props','Language']
country_language=country_language.groupby(['countryCode','name','Language']).size().reset_index(name='COUNT').sort_values(by='name')
country_language=props(country_language,'name')


In [ ]:
#Joining all features together with sampling

data=joining(data,gender_data)
data=joining(data,country_language)
data=joining_states(data,state_data,"state")

In [ ]:
#Dropping of unnecessary columns
data.drop(['props','COUNT'],inplace=True,axis=1)

In [ ]:
# Creating indicators on similar individuals. Assumed that these things aren't allowed to be changed
data.fillna("None",inplace=True) 
data['ind']=data['generation']+' '+data['gender']+' '+data['countryCode']+' '+data['Language']


In [ ]:
#Shuffling of rows within indicators
data=data.sort_values(by=['ind']).groupby(['ind','platform','state'], group_keys=False).apply(lambda x: x.sample(frac=1))

In [ ]:
# Creating ids using the grouped indicators, only allowing people to exist 20-50 times within the table
grouped = data.groupby('ind')
result = []
id = 1
for name, group in grouped:
    curr_group_length=0
    nrow=len(group)
    while curr_group_length<nrow:
        # Randomly allowing a person to exist 20 - 50 times
        repetitions = np.random.randint(20, 51)

        if curr_group_length+repetitions > nrow:
            repetitions=nrow-curr_group_length
        # Repeat the ID repetion times
        repeated_ids = [id] * repetitions

        # Append to the result list
        result.extend(repeated_ids)
        curr_group_length+=repetitions
        # Increment the ID for the next person
        id += 1

result_df = pd.DataFrame(result, columns=['id'])

data = pd.concat([data.reset_index(drop=True), result_df], axis=1)


In [ ]:
# Selecting only the relevant columns for player table
player=data[['id','countryCode','Language','gender','generation']].drop_duplicates()

In [ ]:

# Creating hashed ids
player['hashedId']=player['id'].apply(lambda x:hash_id(x))

In [ ]:
# Creating ratings dictionary with an assumption that Gen-Z Males have the highest rating
rating = {
    'Baby Boomer': {'Female': (20, 4), 'Male': (25, 5), 'Non-Binary': (28, 6), 'None': (26, 5), 'Unknown': (18, 4)},
    'Gen-X': {'Female': (45, 6), 'Male': (50, 5), 'Non-Binary': (55, 7), 'None': (48, 6), 'Unknown': (49, 5)},
    'Millennial': {'Female': (70, 6), 'Male': (75, 5), 'Non-Binary': (78, 7), 'None': (73, 6), 'Unknown': (74, 5)},
    'Gen-Z': {'Female': (80, 7), 'Male': (85, 5), 'Non-Binary': (82, 7), 'None': (83, 6), 'Unknown': (81, 6)},
    'Silent Generation': {'Female': (22, 5), 'Male': (30, 6), 'Non-Binary': (24, 5), 'None': (28, 4), 'Unknown': (20, 3)},
    'Other': {'Female': (65, 6), 'Male': (70, 7), 'Non-Binary': (67, 6), 'None': (68, 6), 'Unknown': (60, 5)},
    'None': {'Female': (50, 6), 'Male': (55, 7), 'Non-Binary': (53, 6), 'None': (52, 5), 'Unknown': (51, 6)}
}



In [ ]:
# Apply function to assign ratings
player['rating'] = player.apply(ratings_generators, axis=1).astype(int)

In [ ]:
# Dictionary for generations to a birth year
generations_to_years = {
    "Silent Generation": (1928, 1945),
    "Baby Boomer": (1946, 1964),
    "Gen-X": (1965, 1980),
    "Millennial": (1981, 1996),
    "Gen-Z": (1997, 2012),
    "Other": None
}


In [ ]:
# Simulating date of births, emails, and arcsId
player['dateOfBirth']=player['generation'].apply(lambda x: generate_birth_date(x))
player['emailAddress']=player['id'].apply(lambda x: fake.email())
player['arcsId']=player['id'].apply(lambda x: fake.uuid4()[:8])


In [ ]:
# Only keeping relevant columns
player=player[['id','hashedId','arcsId','countryCode','generation','dateOfBirth','emailAddress','Language','rating','gender']]

In [ ]:
# Simulating pilStatus
player['pilStatus']=player['id'].apply(lambda x: random.choice([True, False])) 

In [ ]:
# Replacing None string back to None type for proper missing values adjustment
player=player.replace("None",None)

In [ ]:
player.to_csv("player.csv",index=False)